In [ ]:
%load_ext autoreload
%autoreload 2

from IPython import display
from lib.viz import showarray
import matplotlib.pyplot as plt

import pybullet as p
import numpy as np
import cv2

from lib.pyb.pybullet_robot import World, Manipulator, FixedCamera, LinkedCamera

In [ ]:
N = 10
NJ = 4 # number of joint pairs (segments)
NP = 4 # number of plates per segment
NA = 1 # number of axes per segment
PHI_AMP = np.pi/4

w = World(gui=False)
# side camera is on axis X, 2 units away, looking towards zero
side_cam = FixedCamera(w, np.array(((3,0,0.5), (-1,0,0), (0,0,1))))

# zero out angles of all X joints, to enforce movement on YZ plane only
phis = np.random.uniform(low=-PHI_AMP, high=PHI_AMP, size=(N, NJ, 2))
phis[:,:,0] = 0

n_styles = len(Manipulator.STYLES)
side_imgs = np.zeros((n_styles, phis.shape[0], side_cam.H, side_cam.W, 4))
#eye_imgs = np.zeros((n_styles, phis.shape[0], eye_cam.H, eye_cam.W, 4))

for style_idx, style_name in enumerate(Manipulator.STYLES):
    m = Manipulator(w, NJ, NP, NA, style=style_name)
    #eye_cam = LinkedCamera(w, m.body_id, m.eye_link_id)

    for phi_idx in range(phis.shape[0]):
        m.step(phis[phi_idx])
        w.step()

        side_imgs[style_idx][phi_idx] = side_cam.getRGBAImage()
        #eye_imgs[i] = eye_cam.getRGBAImage()

    #eye_cam.close()
    m.close()

side_cam.close()
w.close()

In [ ]:
styles_fname = 'styles.npz'

In [ ]:
np.savez_compressed(styles_fname, phis=phis, side_imgs=side_imgs)

In [ ]:
data = np.load(styles_fname)
print(data['phis'].shape)
print(data['side_imgs'].shape)

In [ ]:
plt.hist(data['phis'][:,:,1].flatten())

In [ ]:
def show_side_imgs(data, N):
    imgs = data['side_imgs']
    
    rows = None
    for i in range(N):
        row = None
        for style_idx, _ in enumerate(Manipulator.STYLES):
            img = imgs[style_idx][i]
            if row is None:
                row = img
            else:
                row = np.hstack((row, img))
        if rows is None:
            rows = row
        else:
            rows = np.vstack((rows, row))

    showarray(rows)

In [ ]:
show_side_imgs(data, 10)

In [ ]:
w = World(gui=True)

In [ ]:
m = Manipulator(w, 4, 4, 4, style=Manipulator.STYLES[0])

In [ ]:
phis = np.random.uniform(low=-PHI_AMP, high=PHI_AMP, size=(4, 2))
phis[:,:] = 0
phis[:,:] = -np.pi/4
m.step(phis)

In [ ]:
m._print_joints_pos()

In [ ]:
m.close()

In [ ]:
w.close()